In [1]:
from dataclasses import dataclass
from enum import StrEnum
from datetime import datetime, timedelta, date
import polars as pl
from collections import defaultdict, Counter
from typing import Tuple
from common.constants.column_types import (
    CPZP_SCHEMA,
    OZP_SCHEMA,
    POHLAVI_CPZP,
    TYP_UDALOSTI,
)
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS, CPZP_COLUMNS
import pickle
from common.constants.objects import (
    Person,
    Gender,
    AgeCohort,
    Prescription,
    PrescriptionType,
)
import matplotlib.pyplot as plt
import numpy as np
import os
from common.utils import (
    draw_chart,
    filter_by_date_range,
)

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(60)

from typing import Any
import matplotlib.dates as mdates
from matplotlib.patches import Patch
from scipy.stats import fisher_exact

plt.rcParams.update(
    {
        "font.size": 14,  # default text size
        "axes.titlesize": 16,  # chart titles
        "axes.labelsize": 14,  # x/y labels
        "xtick.labelsize": 12,  # x-axis tick labels
        "ytick.labelsize": 12,  # y-axis tick labels
        "legend.fontsize": 12,  # legend text
    }
)

In [4]:
# CONSTANTS
POJISTOVNA = "cpzp"
VAX_PERIOD_IN_DAYS = 30

In [5]:
with open(f"./DATACON_data/{POJISTOVNA}_persons.pkl", "rb") as f:
    persons: list[Person] = pickle.load(f)

In [6]:
# VARIABLES
vax_dates_map: dict[AgeCohort, dict[int, list[datetime]]] = defaultdict(dict)
max_vax_intensity_map: dict[AgeCohort, dict[int, datetime]] = defaultdict(dict)

novax_ppl_predpisy_map: dict[AgeCohort, dict[datetime, int]] = defaultdict(
    lambda: defaultdict(int)
)
novax_ppl_prvopredpisy_map: dict[AgeCohort, dict[datetime, int]] = defaultdict(
    lambda: defaultdict(int)
)
novax_ppl_prednison_equivs_map: dict[AgeCohort, dict[datetime, float]] = defaultdict(
    lambda: defaultdict(float)
)
novax_ppl_imunosupresivy_map: dict[AgeCohort, dict[datetime, int]] = defaultdict(
    lambda: defaultdict(int)
)


vax_ppl_prvopredpisy_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
vax_ppl_predpisy_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
vax_ppl_prednison_equivs_map = defaultdict(
    lambda: defaultdict(lambda: defaultdict(int))
)
vax_ppl_imunosupresivy_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

# === MAX VAXINATION DATES BY COHORT ===
for person in persons:
    if person.died_at or not person.vaccines:
        continue
    for vax in person.vaccines:
        if vax.dose_number not in vax_dates_map[vax.age_cohort]:
            vax_dates_map[vax.age_cohort][vax.dose_number] = []
        vax_dates_map[vax.age_cohort][vax.dose_number].append(vax.date)


for age_cohort_at_prescription, doses_map in vax_dates_map.items():
    for dose_number, dates in doses_map.items():
        if not dates:
            continue

        max_date, _ = max(Counter(dates).items(), key=lambda x: x[1])
        max_vax_intensity_map[age_cohort_at_prescription][dose_number] = max_date


# === NOVAX PEOPLE METRICS ===
for p in persons:
    if p.died_at:
        continue

    if not p.vaccines:
        if not p.prescriptions:
            continue

        last_prescription_date: datetime = datetime.min.date()
        for prescription in p.prescriptions:
            age_cohort_at_prescription = prescription.age_cohort_at_prescription
            if (
                prescription.lekova_forma == "Injekční suspenze"
                or prescription.lekova_forma == "Injekční/infuzní roztok"
            ):
                if abs((last_prescription_date - prescription.date).days) < 14:
                    continue
            last_prescription_date = prescription.date

            if prescription.prescription_type == PrescriptionType.IMUNOSUPRESSIVE:
                novax_ppl_imunosupresivy_map[age_cohort_at_prescription][
                    prescription.date
                ] += 1

            novax_ppl_predpisy_map[age_cohort_at_prescription][prescription.date] += 1

            if prescription.prednison_equiv is not None:
                novax_ppl_prednison_equivs_map[age_cohort_at_prescription][
                    prescription.date
                ] += prescription.prednison_equiv

        first_prescription_date = p.prescriptions[0].date
        age_cohort_at_1st_prescription = p.prescriptions[0].age_cohort_at_prescription
        novax_ppl_prvopredpisy_map[age_cohort_at_1st_prescription][
            first_prescription_date
        ] += 1

# === VAX PEOPLE METRICS ===
for p in persons:
    if p.died_at or not p.vaccines:
        continue

    for vax in p.vaccines:
        if not p.prescriptions:
            continue

        max_int_date = max_vax_intensity_map[vax.age_cohort][vax.dose_number]
        if abs((vax.date - max_int_date).days) > VAX_PERIOD_IN_DAYS:
            continue

        # === PRESCRIPTIONS ===
        last_prescription_date: datetime = datetime.min.date()
        for prescription in p.prescriptions:

            if (
                prescription.lekova_forma == "Injekční suspenze"
                or prescription.lekova_forma == "Injekční/infuzní roztok"
            ):
                if abs((last_prescription_date - prescription.date).days) < 14:
                    continue
            last_prescription_date = prescription.date

            relative_prescription_date = (prescription.date - vax.date).days
            if prescription.prescription_type == PrescriptionType.IMUNOSUPRESSIVE:
                vax_ppl_imunosupresivy_map[vax.age_cohort][vax.dose_number][
                    relative_prescription_date
                ] += 1

            # === PRESCRIPTIONS COUNT ===
            vax_ppl_predpisy_map[vax.age_cohort][vax.dose_number][
                relative_prescription_date
            ] += 1

            # === PREDNISON EQUIV ===
            if prescription.prednison_equiv is not None:
                vax_ppl_prednison_equivs_map[vax.age_cohort][vax.dose_number][
                    relative_prescription_date
                ] += prescription.prednison_equiv

        # === FIRST PRESCRIPTIONS ===
        first_prescription = min(p.prescriptions, key=lambda x: x.date)
        relative_first_prescription_date = (first_prescription.date - vax.date).days

        vax_ppl_prvopredpisy_map[vax.age_cohort][vax.dose_number][
            relative_first_prescription_date
        ] += 1

In [7]:
def plot_vax_timeline(vax_dates_map, age_cohort, dose_number):
    df = pl.DataFrame({"date": vax_dates_map[age_cohort][dose_number]})
    counts = df.group_by("date").len().rename({"len": "count"}).sort("date")
    start, end = counts["date"].min(), counts["date"].max()
    counts = (
        pl.DataFrame({"date": pl.date_range(start, end, "1d", eager=True)})
        .join(counts, on="date", how="left")
        .fill_null(0)
        .with_columns(pl.col("count").rolling_mean(7).alias("ma"))
    )

    # peak den
    peak = counts.sort("count", descending=True).head(1)
    peak_date, _ = peak["date"][0], int(peak["count"][0])

    # -30 a +30 dní
    left = peak_date - timedelta(days=30)
    right = peak_date + timedelta(days=30)

    plt.figure(figsize=(14, 6))
    plt.plot(counts["date"], counts["count"], label="Original data", alpha=0.5)
    plt.plot(counts["date"], counts["ma"], label="7-day Moving Average", linewidth=2)

    # peak + popisek
    plt.axvline(
        peak_date,
        color="green",
        linestyle="--",
        linewidth=1.5,
        label=f"Peak: {peak_date}",
    )

    # -30 / +30 dní a vyšrafovaná oblast
    plt.axvline(left, color="green", linestyle="--", linewidth=1)
    plt.axvline(right, color="green", linestyle="--", linewidth=1)
    plt.axvspan(
        left, right, facecolor="green", alpha=0.08, hatch="//", edgecolor="green"
    )

    plt.title(f"Vaccination Timeline - {age_cohort} - Dose {dose_number}")
    plt.xlabel("Date")  # klidně změním na denní offsety
    plt.ylabel("Number of vaccinations")
    plt.legend()
    plt.tight_layout()
    os.makedirs(f"out/{POJISTOVNA}/vax_period", exist_ok=True)
    plt.savefig(
        f"out/{POJISTOVNA}/vax_period/{age_cohort.value}-dose_{dose_number}.png"
    )
    plt.close()


for age_cohort in AgeCohort:
    for dose_number in [1, 2, 3]:
        plot_vax_timeline(vax_dates_map, age_cohort, dose_number)

In [ ]:
class ChartDrawer:
    def draw_2x2_block(
        self,
        vax_dates_map,
        novax_dates_map,
        rozhodne_datum,
        title,
        axes,
        row_offset,
        col_offset,
    ):
        vax_sums = self.__get_before_after_sums(vax_dates_map, rozhodne_datum)
        novax_sums = self.__get_before_after_sums(novax_dates_map, rozhodne_datum)
        total_vax_sum = sum(vax_sums.values())
        total_novax_sum = sum(novax_sums.values())

        # Očkovaná skupina
        self.__draw_scatter_plot(
            axes[row_offset][col_offset],
            list(vax_dates_map.keys()),
            list(vax_dates_map.values()),
            f"{title} - Očkovaná skupina",
            rozhodne_datum,
            total_vax_sum,
        )

        self.__draw_bar_chart(
            axes[row_offset][col_offset + 1],
            list(vax_sums.keys()),
            list(vax_sums.values()),
            f"{title} - Očkovaná skupina",
        )

        # Neočkovaná skupina
        self.__draw_scatter_plot(
            axes[row_offset + 1][col_offset],
            list(novax_dates_map.keys()),
            list(novax_dates_map.values()),
            f"{title} - Neočkovaná skupina",
            rozhodne_datum,
            total_novax_sum,
        )

        self.__draw_bar_chart(
            axes[row_offset + 1][col_offset + 1],
            list(novax_sums.keys()),
            list(novax_sums.values()),
            f"{title} - Neočkovaná skupina",
        )

    def __draw_scatter_plot(self, ax, x_data, y_data, title, rozhodne_datum, total_sum):
        ax.plot(x_data, y_data, label="Data", alpha=0.7, marker="o", linestyle="None")

        # --- Bucket by weeks (7-day bins) ---
        weekly_buckets = defaultdict(list)
        for x, y in zip(x_data, y_data):
            # integer division to assign week bin
            week_start = (x // 7) * 7
            weekly_buckets[week_start].append(y)

        # Compute weekly averages
        weekly_avg_points = sorted(
            (week_start, sum(vals) / len(vals))
            for week_start, vals in weekly_buckets.items()
        )
        avg_x, avg_y = zip(*weekly_avg_points)

        # --- Draw line for weekly averages ---
        ax.plot(avg_x, avg_y, linewidth=2, marker="o", label="Týdenní průměr")

        # rozhodné datum (vypadá jako int, takže taky použijeme přímku)
        ax.axvline(
            x=rozhodne_datum,
            color="green",
            linestyle="--",
            linewidth=2,
        )

        # průměry před/po
        before_values = [y for x, y in zip(x_data, y_data) if x < rozhodne_datum]
        if before_values:
            before_avg = sum(before_values) / len(before_values)
            ax.axhline(before_avg, color="blue", linestyle="--", label="Průměr před")

        after_values = [y for x, y in zip(x_data, y_data) if x >= rozhodne_datum]
        if after_values:
            after_avg = sum(after_values) / len(after_values)
            ax.axhline(after_avg, color="purple", linestyle="--", label="Průměr po")

        ax.tick_params(axis="x", rotation=45)
        ax.set_xlabel("Dny kolem max intenzity")
        ax.set_ylabel("Počet předpisů (týdenní průměr)")
        ax.set_title(title)
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.text(
            0.99,
            0.95,
            f"Celkem: {total_sum:,}",
            transform=ax.transAxes,
            ha="right",
            va="top",
            fontsize=10,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.5),
        )

    def __draw_bar_chart(self, ax, x_data, y_data, title):
        bars = ax.bar(
            x_data,
            y_data,
            color="skyblue",
            edgecolor="black",
            width=0.4,
        )
        ax.set_xlabel("Období před a po")
        ax.set_ylabel("Celkový počet předpisů")
        ax.set_title(title)
        ax.grid(True, alpha=0.3)

        percentages = [100, (y_data[1] / y_data[0]) * 100 if y_data[0] != 0 else 0]
        for i, (bar, value) in enumerate(zip(bars, y_data)):
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                bar.get_height(),
                f"{value:,} / {percentages[i]:.2f}%",
                ha="center",
                va="bottom",
                fontsize=12,
            )

    def __get_before_after_sums(self, dates_map, rozhodne_datum):
        before_items = [(d, v) for d, v in dates_map.items() if d < rozhodne_datum]
        after_items = [(d, v) for d, v in dates_map.items() if d >= rozhodne_datum]
        before_sum = sum(v for _, v in before_items)
        after_sum = sum(v for _, v in after_items)
        return {"před": before_sum, "po": after_sum}

    def draw_vax_vs_unvax_sums(
        self, ax, vax_dates_map, novax_dates_map, rozhodne_datum, title
    ):
        vax = self.__get_before_after_sums(vax_dates_map, rozhodne_datum)
        novax = self.__get_before_after_sums(novax_dates_map, rozhodne_datum)

        x_labels = [
            "Očkovaná: před",
            "Očkovaná: po",
            "Neočkovaná: před",
            "Neočkovaná: po",
        ]
        y_values = [vax["před"], vax["po"], novax["před"], novax["po"]]
        colors = ["skyblue", "skyblue", "seagreen", "seagreen"]

        bars = ax.bar(
            x_labels,
            y_values,
            color=colors,
            edgecolor="black",
            width=1,
        )

        vax_percentages = [
            100,
            (vax["po"] / vax["před"]) * 100 if vax["před"] != 0 else 0,
        ]
        novax_percentages = [
            100,
            (novax["po"] / novax["před"]) * 100 if novax["před"] != 0 else 0,
        ]
        percentages = [
            vax_percentages[0],
            vax_percentages[1],
            novax_percentages[0],
            novax_percentages[1],
        ]

        for i, (bar, value) in enumerate(zip(bars, y_values)):
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                bar.get_height(),
                f"{value:,} / {percentages[i]:.2f}%",
                ha="center",
                va="bottom",
                fontsize=14,
                fontweight="bold",
            )

        ax.set_title(title)
        ax.grid(True, axis="y", alpha=0.3)
        ax.tick_params(axis="x", rotation=15)

        legend_handles = [
            Patch(facecolor="skyblue", edgecolor="black", label="Očkovaná skupina"),
            Patch(facecolor="seagreen", edgecolor="black", label="Neočkovaná skupina"),
        ]
        ax.legend(handles=legend_handles, loc="best")

In [7]:
def get_before_after_sums(dates_map, rozhodne_datum=0):
    before_items = [(d, v) for d, v in dates_map.items() if d < rozhodne_datum]
    after_items = [(d, v) for d, v in dates_map.items() if d >= rozhodne_datum]

    before_sum = sum(v for _, v in before_items)
    after_sum = sum(v for _, v in after_items)

    return {"před": before_sum, "po": after_sum}

In [9]:
for PRESCRIPTION_PERIOD_IN_DAYS in [30, 60, 90, 180, 364]:
    for age_cohort_at_prescription in AgeCohort:
        for dose_number in [1, 2, 3]:
            rozhodne_datum = max_vax_intensity_map[age_cohort_at_prescription][
                dose_number
            ]

            def filter_current_novax_cohort(dates_map) -> dict[int, int]:
                start = rozhodne_datum - timedelta(days=PRESCRIPTION_PERIOD_IN_DAYS)
                end = rozhodne_datum + timedelta(days=PRESCRIPTION_PERIOD_IN_DAYS)
                return {
                    (day - rozhodne_datum)
                    .days: dates_map[age_cohort_at_prescription]
                    .get(day, 0)
                    for day in (
                        start + timedelta(days=i) for i in range((end - start).days + 1)
                    )
                }

            def filter_current_vax_cohort(dates_map) -> dict[int, int]:
                return {
                    day: dates_map[age_cohort_at_prescription][dose_number].get(day, 0)
                    for day in range(
                        -PRESCRIPTION_PERIOD_IN_DAYS, PRESCRIPTION_PERIOD_IN_DAYS
                    )
                }

            vax_predpisy_map = filter_current_vax_cohort(vax_ppl_predpisy_map)
            novax_predpisy_map = filter_current_novax_cohort(novax_ppl_predpisy_map)

            vax_prvopredpisy_map = filter_current_vax_cohort(vax_ppl_prvopredpisy_map)
            novax_prvopredpisy_map = filter_current_novax_cohort(
                novax_ppl_prvopredpisy_map
            )

            vax_kortikoidy_map = filter_current_vax_cohort(vax_ppl_prednison_equivs_map)
            novax_kortikoidy_map = filter_current_novax_cohort(
                novax_ppl_prednison_equivs_map
            )

            vax_imunosupresivy_map = filter_current_vax_cohort(
                vax_ppl_imunosupresivy_map
            )
            novax_imunosupresivy_map = filter_current_novax_cohort(
                novax_ppl_imunosupresivy_map
            )

            drawer = ChartDrawer()

            fig, axes = plt.subplots(nrows=2, ncols=8, figsize=(80, 20))
            fig.tight_layout(pad=10.0)
            fig.suptitle(
                f" {POJISTOVNA} - {PRESCRIPTION_PERIOD_IN_DAYS} Dnů od {rozhodne_datum.strftime('%d.%m.%Y')} - {age_cohort_at_prescription.value} - Dose {dose_number}",
                fontsize=36,
            )

            drawer.draw_2x2_block(
                vax_dates_map=vax_predpisy_map,
                novax_dates_map=novax_predpisy_map,
                rozhodne_datum=0,
                title="Předpisy",
                axes=axes,
                row_offset=0,
                col_offset=0,
            )
            drawer.draw_2x2_block(
                vax_dates_map=vax_prvopredpisy_map,
                novax_dates_map=novax_prvopredpisy_map,
                rozhodne_datum=0,
                title="Prvopředpisy",
                axes=axes,
                row_offset=0,
                col_offset=2,
            )
            drawer.draw_2x2_block(
                vax_dates_map=vax_kortikoidy_map,
                novax_dates_map=novax_kortikoidy_map,
                rozhodne_datum=0,
                title="Kortikoidové ekvivalenty",
                axes=axes,
                row_offset=0,
                col_offset=4,
            )

            drawer.draw_2x2_block(
                vax_dates_map=vax_imunosupresivy_map,
                novax_dates_map=novax_imunosupresivy_map,
                rozhodne_datum=0,
                title="Imunosupresivní předpisy",
                axes=axes,
                row_offset=0,
                col_offset=6,
            )

            # fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(40, 10))
            # fig.tight_layout(pad=10.0)
            # fig.suptitle(
            #     f" {POJISTOVNA} - {PRESCRIPTION_PERIOD_IN_DAYS} Dnů od {rozhodne_datum.strftime('%d.%m.%Y')} - {age_cohort_at_prescription.value} - Dose {dose_number}",
            #     fontsize=36,
            # )

            # drawer.draw_vax_vs_unvax_sums(
            #     ax=axes[0],
            #     vax_dates_map=vax_predpisy_map,
            #     novax_dates_map=novax_predpisy_map,
            #     rozhodne_datum=0,
            #     title="Předpisy",
            # )

            # drawer.draw_vax_vs_unvax_sums(
            #     ax=axes[1],
            #     vax_dates_map=vax_prvopredpisy_map,
            #     novax_dates_map=novax_prvopredpisy_map,
            #     rozhodne_datum=0,
            #     title="Prvopředpisy",
            # )

            # drawer.draw_vax_vs_unvax_sums(
            #     ax=axes[2],
            #     vax_dates_map=vax_kortikoidy_map,
            #     novax_dates_map=novax_kortikoidy_map,
            #     rozhodne_datum=0,
            #     title="Kortikoidové ekvivalenty",
            # )

            # drawer.draw_vax_vs_unvax_sums(
            #     ax=axes[3],
            #     vax_dates_map=vax_imunosupresivy_map,
            #     novax_dates_map=novax_imunosupresivy_map,
            #     rozhodne_datum=0,
            #     title="Imunosupresivní předpisy",
            # )

            os.makedirs(
                f"out/{POJISTOVNA}/2x2/{PRESCRIPTION_PERIOD_IN_DAYS}", exist_ok=True
            )
            plt.savefig(
                f"out/{POJISTOVNA}/2x2/{PRESCRIPTION_PERIOD_IN_DAYS}/{age_cohort_at_prescription.value}-dose_{dose_number}.png"
            )
            plt.close(fig)

In [10]:
import pandas as pd

PRESCRIPTION_PERIOD_IN_DAYS = 180
rows: list[dict] = []


def _safe_div(a, b):
    return np.nan if b == 0 else (a / b)


def _pvalue_from_df(df: pd.DataFrame) -> float:
    # df index: ["očkovaní","neočkovaní"], columns: ["před","po"]
    table = [
        [int(df.loc["očkovaní", "před"]), int(df.loc["očkovaní", "po"])],
        [int(df.loc["neočkovaní", "před"]), int(df.loc["neočkovaní", "po"])],
    ]
    return fisher_exact(table, alternative="two-sided")[1]


for age_cohort_at_prescription in AgeCohort:
    for dose_number in [1, 2, 3]:
        rozhodne_datum = max_vax_intensity_map[age_cohort_at_prescription][dose_number]

        def filter_current_novax_cohort(dates_map) -> dict[int, int]:
            start = rozhodne_datum - timedelta(days=PRESCRIPTION_PERIOD_IN_DAYS)
            end = rozhodne_datum + timedelta(days=PRESCRIPTION_PERIOD_IN_DAYS)
            return {
                (day - rozhodne_datum)
                .days: dates_map[age_cohort_at_prescription]
                .get(day, 0)
                for day in (
                    start + timedelta(days=i) for i in range((end - start).days + 1)
                )
            }

        def filter_current_vax_cohort(dates_map) -> dict[int, int]:
            return {
                day: dates_map[age_cohort_at_prescription][dose_number].get(day, 0)
                for day in range(
                    -PRESCRIPTION_PERIOD_IN_DAYS, PRESCRIPTION_PERIOD_IN_DAYS
                )
            }

        # build per-metric before/after DFs (očkovaní/neočkovaní × před/po)
        def get_before_after_df(vax_map, novax_map) -> pd.DataFrame:
            vax_before_after = get_before_after_sums(vax_map)
            novax_before_after = get_before_after_sums(novax_map)
            return pd.DataFrame(
                {
                    "před": [vax_before_after["před"], novax_before_after["před"]],
                    "po": [vax_before_after["po"], novax_before_after["po"]],
                },
                index=["očkovaní", "neočkovaní"],
            )

        vax_predpisy_map = filter_current_vax_cohort(vax_ppl_predpisy_map)
        novax_predpisy_map = filter_current_novax_cohort(novax_ppl_predpisy_map)

        vax_prvopredpisy_map = filter_current_vax_cohort(vax_ppl_prvopredpisy_map)
        novax_prvopredpisy_map = filter_current_novax_cohort(novax_ppl_prvopredpisy_map)

        vax_kortikoidy_map = filter_current_vax_cohort(vax_ppl_prednison_equivs_map)
        novax_kortikoidy_map = filter_current_novax_cohort(
            novax_ppl_prednison_equivs_map
        )

        vax_imunosupresivy_map = filter_current_vax_cohort(vax_ppl_imunosupresivy_map)
        novax_imunosupresivy_map = filter_current_novax_cohort(
            novax_ppl_imunosupresivy_map
        )

        # per-metric DFs
        predpisy_df = get_before_after_df(vax_predpisy_map, novax_predpisy_map)
        prvopred_df = get_before_after_df(vax_prvopredpisy_map, novax_prvopredpisy_map)
        kortik_df = get_before_after_df(vax_kortikoidy_map, novax_kortikoidy_map)
        imun_df = get_before_after_df(vax_imunosupresivy_map, novax_imunosupresivy_map)

        for metric_name, df in [
            ("predpisy", predpisy_df),
            ("prvopredpisy", prvopred_df),
            ("kortikoidy", kortik_df),
            ("imunosupresivy", imun_df),
        ]:
            vax_before = float(df.loc["očkovaní", "před"])
            vax_after = float(df.loc["očkovaní", "po"])
            novax_before = float(df.loc["neočkovaní", "před"])
            novax_after = float(df.loc["neočkovaní", "po"])

            vax_increase = (
                _safe_div(vax_after, vax_before) * 100 if vax_before != 0 else np.nan
            )
            novax_increase = (
                _safe_div(novax_after, novax_before) * 100
                if novax_before != 0
                else np.nan
            )
            diff = (
                (vax_increase - novax_increase)
                if (np.isfinite(vax_increase) and np.isfinite(novax_increase))
                else np.nan
            )
            p_value = _pvalue_from_df(df)

            # optional: ratio of the increases (useful effect size >1 means větší relativní nárůst u očkovaných)
            vax_vs_novax_ratio = np.nan
            if (
                vax_before != 0
                and novax_before != 0
                and vax_increase
                and novax_increase
            ):
                vax_vs_novax_ratio = _safe_div(
                    vax_after / vax_before, novax_after / novax_before
                )

            rows.append(
                {
                    "age_cohort": str(age_cohort_at_prescription),
                    "vax_dose": int(dose_number),
                    "metric": metric_name,
                    "vax_increase": vax_increase,
                    "novax_increase": novax_increase,
                    "diff": diff,
                    "p_value": p_value,
                    "vax_before": vax_before,
                    "vax_after": vax_after,
                    "novax_before": novax_before,
                    "novax_after": novax_after,
                }
            )

# finální “velký” DataFrame
huge_df = pd.DataFrame(
    rows,
    columns=[
        "age_cohort",
        "vax_dose",
        "metric",
        "vax_increase",
        "novax_increase",
        "diff",
        "p_value",
        "vax_before",
        "vax_after",
        "novax_before",
        "novax_after",
    ],
)

# pokud chceš hezčí pořadí a typy:
huge_df["vax_dose"] = huge_df["vax_dose"].astype("int64")
huge_df = huge_df.sort_values(["metric", "age_cohort", "vax_dose"]).reset_index(
    drop=True
)

huge_df.to_csv(f"out/{POJISTOVNA}/results.csv", index=False)
print(huge_df)

           age_cohort  vax_dose          metric  vax_increase  novax_increase  \
0   between_12_and_30         1  imunosupresivy    119.710145      111.732852   
1   between_12_and_30         2  imunosupresivy    113.970588      113.898917   
2   between_12_and_30         3  imunosupresivy    108.400000      109.609121   
3   between_30_and_50         1  imunosupresivy     99.214976      105.417607   
4   between_30_and_50         2  imunosupresivy    103.596350      103.646409   
5   between_30_and_50         3  imunosupresivy    102.933673      111.620469   
6   between_50_and_70         1  imunosupresivy    101.348315       96.416938   
7   between_50_and_70         2  imunosupresivy    104.747320      104.377104   
8   between_50_and_70         3  imunosupresivy    107.125104      119.743178   
9        less_than_12         1  imunosupresivy    166.666667       77.067669   
10       less_than_12         2  imunosupresivy    333.333333       87.500000   
11       less_than_12       